In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [5]:
np.random.seed(0)
data = pd.read_csv("..\\Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("..\\Data\\cle\\Rand_valid.csv", sep=',')

In [6]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X
X2_val = X_val

In [7]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (1-(Y <= cut) * 1) * Y
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (1-(Y_val <= cut) * 1) * Y_val
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [8]:
from fit9_nn import fit, comb_error , mae_error

In [9]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias = True)
        self.fc12 = nn.Linear(24, 1, bias = True)
       
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 1, bias=True)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = torch.sigmoid(self.fc12(x1))
        
        x2 = self.fc21(x)
        x2 = torch.tanh(self.fc22(x2))
        x2 = torch.exp(self.fc23(x2))       
        
        x3 = torch.cat((x1, x2), 1)
        return x3

In [10]:
n_epochs = 2000
lr = 0.0001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 5
clipping = 0.20
sig2 = 2 * 950000
PATH = '..\\Data\\checkpoints\\mytraining2.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [12]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [13]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.data.uniform_(-y, y)

In [14]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = 'Data\\checkpoints\\mytraining2.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, val_error = mae_error,
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)

Epoch 1: Training Loss: 0.9879429737726847, Validation Loss: 289.8795166015625
Epoch 2: Training Loss: 0.9858556687831879, Validation Loss: 289.87213134765625
Epoch 3: Training Loss: 0.983793705701828, Validation Loss: 289.86431884765625
Epoch 4: Training Loss: 0.9817736546198527, Validation Loss: 289.856201171875
Epoch 5: Training Loss: 0.979804515838623, Validation Loss: 289.8475646972656
Epoch 6: Training Loss: 0.9778568844000498, Validation Loss: 289.8382568359375
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 0.9759786625703176, Validation Loss: 289.82830810546875
Epoch 8: Training Loss: 0.9740764002005259, Validation Loss: 289.8172607421875
Epoch 9: Training Loss: 0.9722557067871094, Validation Loss: 289.8050842285156
Epoch 10: Training Loss: 0.9704098006089529, Validation Loss: 289.7913818359375
Epoch 11: Training Loss: 0.9686348140239716, Validation Loss: 289.7760925292969
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 0.9668897787729899, Validation Loss: 289.7

Epoch 97: Training Loss: 0.8880802591641744, Validation Loss: 285.93621826171875
Epoch 98: Training Loss: 0.8875383734703064, Validation Loss: 285.92083740234375
Epoch 99: Training Loss: 0.8870251476764679, Validation Loss: 285.90570068359375
Epoch 100: Training Loss: 0.8864892224470774, Validation Loss: 285.89031982421875
Epoch 101: Training Loss: 0.8859834770361582, Validation Loss: 285.875
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.885470062494278, Validation Loss: 285.8597412109375
Epoch 103: Training Loss: 0.8849314550558726, Validation Loss: 285.84423828125
Epoch 104: Training Loss: 0.8844446440537771, Validation Loss: 285.8292541503906
Epoch 105: Training Loss: 0.8839387396971384, Validation Loss: 285.8140869140625
Epoch 106: Training Loss: 0.883444219827652, Validation Loss: 285.7989807128906
New Checkpoint Saved into PATH
Epoch 107: Training Loss: 0.8829479217529297, Validation Loss: 285.7839050292969
Epoch 108: Training Loss: 0.882445216178894, Validation Loss

Epoch 193: Training Loss: 0.8568011919657389, Validation Loss: 284.3633728027344
Epoch 194: Training Loss: 0.8566819230715433, Validation Loss: 284.3468322753906
Epoch 195: Training Loss: 0.8565602600574493, Validation Loss: 284.3302307128906
Epoch 196: Training Loss: 0.8564551472663879, Validation Loss: 284.3138122558594
New Checkpoint Saved into PATH
Epoch 197: Training Loss: 0.8563453455766042, Validation Loss: 284.2979431152344
Epoch 198: Training Loss: 0.8562427361806234, Validation Loss: 284.2806396484375
Epoch 199: Training Loss: 0.8561348915100098, Validation Loss: 284.2644348144531
Epoch 200: Training Loss: 0.856038510799408, Validation Loss: 284.24761962890625
Epoch 201: Training Loss: 0.8559393783410391, Validation Loss: 284.2303771972656
New Checkpoint Saved into PATH
Epoch 202: Training Loss: 0.8558343350887299, Validation Loss: 284.2137145996094
Epoch 203: Training Loss: 0.855741282304128, Validation Loss: 284.19696044921875
Epoch 204: Training Loss: 0.8556424776713053, V

Epoch 287: Training Loss: 0.850642571846644, Validation Loss: 282.8174133300781
Epoch 288: Training Loss: 0.8505923847357432, Validation Loss: 282.80059814453125
Epoch 289: Training Loss: 0.8505490223566691, Validation Loss: 282.7830505371094
Epoch 290: Training Loss: 0.850506434837977, Validation Loss: 282.76556396484375
Epoch 291: Training Loss: 0.8504634201526642, Validation Loss: 282.7481994628906
New Checkpoint Saved into PATH
Epoch 292: Training Loss: 0.8504179914792379, Validation Loss: 282.7310791015625
Epoch 293: Training Loss: 0.8503784040609995, Validation Loss: 282.71337890625
Epoch 294: Training Loss: 0.8503361543019613, Validation Loss: 282.6964111328125
Epoch 295: Training Loss: 0.8502941032250723, Validation Loss: 282.6800842285156
Epoch 296: Training Loss: 0.850247452656428, Validation Loss: 282.66339111328125
New Checkpoint Saved into PATH
Epoch 297: Training Loss: 0.8502077460289001, Validation Loss: 282.646728515625
Epoch 298: Training Loss: 0.8501687546571096, Vali

Epoch 383: Training Loss: 0.8469437062740326, Validation Loss: 281.0912780761719
Epoch 384: Training Loss: 0.8469089170296987, Validation Loss: 281.07183837890625
Epoch 385: Training Loss: 0.8468693494796753, Validation Loss: 281.0526123046875
Epoch 386: Training Loss: 0.8468368848164877, Validation Loss: 281.0347900390625
New Checkpoint Saved into PATH
Epoch 387: Training Loss: 0.8467975854873657, Validation Loss: 281.0166931152344
Epoch 388: Training Loss: 0.8467614154020945, Validation Loss: 280.99737548828125
Epoch 389: Training Loss: 0.8467259705066681, Validation Loss: 280.9783630371094
Epoch 390: Training Loss: 0.8466910024483999, Validation Loss: 280.95904541015625
Epoch 391: Training Loss: 0.8466517925262451, Validation Loss: 280.9396667480469
New Checkpoint Saved into PATH
Epoch 392: Training Loss: 0.8466152946154276, Validation Loss: 280.9203796386719
Epoch 393: Training Loss: 0.8465820749600729, Validation Loss: 280.89971923828125
Epoch 394: Training Loss: 0.846541742483774

...

Epoch 1569: Training Loss: 0.7969219088554382, Validation Loss: 311.2483215332031
Epoch 1570: Training Loss: 0.7968882620334625, Validation Loss: 311.1606140136719
Epoch 1571: Training Loss: 0.796856701374054, Validation Loss: 311.4503173828125
Epoch 1572: Training Loss: 0.7968201736609141, Validation Loss: 311.7088317871094
Epoch 1573: Training Loss: 0.7967826128005981, Validation Loss: 311.877685546875
Epoch 1574: Training Loss: 0.796752393245697, Validation Loss: 311.6398620605469
Epoch 1575: Training Loss: 0.7967169185479482, Validation Loss: 311.82562255859375
Epoch 1576: Training Loss: 0.7966879109541575, Validation Loss: 311.41876220703125
Epoch 1577: Training Loss: 0.7966429988543192, Validation Loss: 311.42840576171875
Epoch 1578: Training Loss: 0.79661292831103, Validation Loss: 311.3727111816406
Epoch 1579: Training Loss: 0.7965758144855499, Validation Loss: 311.5047302246094
Epoch 1580: Training Loss: 0.7965459326903025, Validation Loss: 311.80609130859375
Epoch 1581: Train

Epoch 1669: Training Loss: 0.7937730451424917, Validation Loss: 314.13623046875
Epoch 1670: Training Loss: 0.7937437295913696, Validation Loss: 314.4397888183594
Epoch 1671: Training Loss: 0.7937094271183014, Validation Loss: 314.485595703125
Epoch 1672: Training Loss: 0.7936865786711375, Validation Loss: 313.9391784667969
Epoch 1673: Training Loss: 0.793651153643926, Validation Loss: 313.87353515625
Epoch 1674: Training Loss: 0.7936307986577352, Validation Loss: 313.7331848144531
Epoch 1675: Training Loss: 0.7935958007971445, Validation Loss: 313.8642883300781
Epoch 1676: Training Loss: 0.7935695548852285, Validation Loss: 314.0760803222656
Epoch 1677: Training Loss: 0.793558806180954, Validation Loss: 314.5542907714844
Epoch 1678: Training Loss: 0.7935172518094381, Validation Loss: 314.6947937011719
Epoch 1679: Training Loss: 0.793491393327713, Validation Loss: 314.790771484375
Epoch 1680: Training Loss: 0.7934800485769907, Validation Loss: 314.36724853515625
Epoch 1681: Training Los

Epoch 1771: Training Loss: 0.791176031033198, Validation Loss: 314.4504089355469
Epoch 1772: Training Loss: 0.791164626677831, Validation Loss: 314.6942138671875
Epoch 1773: Training Loss: 0.7911253869533539, Validation Loss: 314.2969970703125
Epoch 1774: Training Loss: 0.7911104261875153, Validation Loss: 313.5962219238281
Epoch 1775: Training Loss: 0.7910811305046082, Validation Loss: 313.9443054199219
Epoch 1776: Training Loss: 0.7910559276739756, Validation Loss: 313.968505859375
Epoch 1777: Training Loss: 0.7910365959008535, Validation Loss: 313.912109375
Epoch 1778: Training Loss: 0.7910090784231821, Validation Loss: 313.925048828125
Epoch 1779: Training Loss: 0.7909861902395884, Validation Loss: 314.4130859375
Epoch 1780: Training Loss: 0.7909610470136007, Validation Loss: 314.8082580566406
Epoch 1781: Training Loss: 0.7909363309542338, Validation Loss: 314.9046936035156
Epoch 1782: Training Loss: 0.7909093995889028, Validation Loss: 314.9141845703125
Epoch 1783: Training Loss: 

Epoch 1873: Training Loss: 0.7888132433096567, Validation Loss: 313.2962951660156
Epoch 1874: Training Loss: 0.7887960076332092, Validation Loss: 313.3494567871094
Epoch 1875: Training Loss: 0.7887775699297587, Validation Loss: 313.2450866699219
Epoch 1876: Training Loss: 0.7887541403373083, Validation Loss: 313.31561279296875
Epoch 1877: Training Loss: 0.7887443602085114, Validation Loss: 313.7883605957031
Epoch 1878: Training Loss: 0.7887082099914551, Validation Loss: 313.730712890625
Epoch 1879: Training Loss: 0.7886882722377777, Validation Loss: 313.3465881347656
Epoch 1880: Training Loss: 0.7886664470036825, Validation Loss: 313.3418273925781
Epoch 1881: Training Loss: 0.7886379361152649, Validation Loss: 313.38409423828125
Epoch 1882: Training Loss: 0.7886219223340353, Validation Loss: 313.8142395019531
Epoch 1883: Training Loss: 0.7885980804761251, Validation Loss: 313.5105285644531
Epoch 1884: Training Loss: 0.7885742783546448, Validation Loss: 313.5931396484375
Epoch 1885: Tra

Epoch 1973: Training Loss: 0.7865826686223348, Validation Loss: 313.0280456542969
Epoch 1974: Training Loss: 0.7865455448627472, Validation Loss: 312.958740234375
Epoch 1975: Training Loss: 0.7865232825279236, Validation Loss: 312.7322998046875
Epoch 1976: Training Loss: 0.7865053117275238, Validation Loss: 312.8611145019531
Epoch 1977: Training Loss: 0.7864823639392853, Validation Loss: 312.4337463378906
Epoch 1978: Training Loss: 0.7864648501078287, Validation Loss: 311.9369201660156
Epoch 1979: Training Loss: 0.7864352365334829, Validation Loss: 311.7313232421875
Epoch 1980: Training Loss: 0.7864253173271815, Validation Loss: 311.18572998046875
Epoch 1981: Training Loss: 0.7863912781079611, Validation Loss: 311.3664855957031
Epoch 1982: Training Loss: 0.7863743007183075, Validation Loss: 311.7768249511719
Epoch 1983: Training Loss: 0.7863433162371317, Validation Loss: 312.2514953613281
Epoch 1984: Training Loss: 0.7863468627134959, Validation Loss: 313.11614990234375
Epoch 1985: Tra